<a href="https://colab.research.google.com/github/mitchelljphayes/COMP9444-group-assignment/blob/main/nx_base_FALCON_7Binstruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate datasets bitsandbytes einops wandb bert_score
import os
import logging
# import transformers
import bert_score
import pandas as pd
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from bert_score import score, BERTScorer
from datasets import load_dataset
import matplotlib.pyplot as plt
from matplotlib import rcParams

# transformers.tokenization_utils.logger.setLevel(logging.ERROR)
# transformers.configuration_utils.logger.setLevel(logging.ERROR)
# transformers.modeling_utils.logger.setLevel(logging.ERROR)

In [ ]:
%matplotlib inline
bert_score.__version__

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import transformers
import torch

# model = "mrm8488/falcoder-7b"
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
from datasets import load_dataset

dataset_name = "mjphayes/machine_learning_questions"
dataset = load_dataset(dataset_name, split="train")

In [ ]:
dataset.shape

In [ ]:
dataset[0]['question']

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

In [ ]:
questions = [i['question'] for i in dataset]

# Batch processing
batch_size = 8  # Adjust as needed
batches = [questions[i:i+batch_size] for i in range(0, len(questions), batch_size)]

# Initialize an empty DataFrame to store results
result = pd.DataFrame(columns=['questions', 'answer', 'model_answer'])

# Process batches
for batch in batches:
    gens = pipeline(
        batch,
        max_length=200,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )

    for i, gen in enumerate(gens):
        result.loc[len(result)] = [batch[i], dataset[i]['answer'], gen[0]['generated_text']]


In [ ]:
result_b = result.copy()

In [ ]:
result_b.to_csv('nx_7binstruct_base_trainset_output.csv',index=True)

STOP HERE

gen2

In [ ]:
dataset[0]['__index_level_0__']
result = pd.DataFrame(columns=['questions','answer','model_answer'])
for i in dataset:
  result.loc[i['__index_level_0__']] = [i['question'],i['answer'],str(i['__index_level_0__'])]

In [ ]:
result.shape

In [ ]:
result = pd.DataFrame(columns=['questions','answer','model_answer'])
# for i in dataset:
#   result.loc[i['__index_level_0__']] = [i['question'],i['answer'],str(i['__index_level_0__'])]

for i in dataset:
  prompt = i['question']
  gen = pipeline(
    prompt,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
  )
  result.loc[i['__index_level_0__']] = [i['question'],i['answer'],gen[0]['generated_text']]
  # result.loc[i['__index_level_0__']] = [i['question'],i['answer'],str(i['__index_level_0__'])]



In [ ]:
result_b = result.copy()

In [ ]:
result

In [ ]:
result_b.to_csv('nx_7binstruct_base_trainset_output1.csv',index=True)

if model_answer still includes the q

In [ ]:
result['model_answer'][2].split('\n')[1]

In [ ]:
range(len(result))

In [ ]:
result_sanitised = result.copy()

In [ ]:
result_sanitised['model_answer'] = result_sanitised['model_answer'].apply(lambda x: x.split('\n')[1] if '\n' in x else x)

In [ ]:
result_sanitised[2:3]['model_answer']

In [ ]:
result_sanitised.to_csv('nx_7binstruct_baseline_output3.csv', index=True)

run eval

In [ ]:
eval_set = pd.read_csv('nx_7binstruct_baseline_output2.csv')

In [ ]:
ideal_answers = eval_set['answer']
gen_answers = eval_set['model_answer']

In [ ]:
ideal_answers

In [ ]:
P, R, F1 = score(gen_answers, ideal_answers, lang='en', verbose=True)

In [ ]:
print(f"System level F1 score: {F1.mean():.3f}")